In [1]:
import csv

from astropy.io import ascii
from astropy.coordinates import SkyCoord, Angle
from astropy.io import fits, votable
import numpy as np
from astropy import units as u
from astropy.table import Table

## Prepare source cutout run

This notebook sets up the parameters for bulk source cutouts.

It will create the following files which can be used as input by askap_cutout_daemon.py to produce fits cutouts around the target sources:
* sb10941_srcs_image_params.vot
* targets_10941.csv

It takes the beam listings as input, e.g. beam_listing_SB10941.csv as well as the Selavy continuum source catalogue for the field. The two output files contain the selected source details (name, location) and the beams that are close to the source and should be used to produce the cutout cubelet. The names of the output files are based on the input ASKAP sbid.

Note this code assumes the use of a 36 beam pattern with 3 interleaves, so a total of 108 beams. It also assumes only one set of beams. If multiple observations are to be included, then the measurements sets for all observations must be concatenated so there is a single measurement set per beam.

### Cutout process

askap_cutout_daemon.py should be run on the login node of a cluster (e.g. avatar). It will manage a series of jobs, one for each source, keeping a defined number running at a time. Each job will use CASA to produce a cubelet around the source based on the beams that are close enough to the source to be provide data for the source.

Each job runs make_askap_abs_cutout.sh to start CASA to run the sub_cube_abs.py script. This has the CASA commands to produce the cubelet. It will create a cubelet covering just the immediate region f the source, it will exclude short baselines, to limit the large scale emission included, and will not do any cleaning.

### Input data

The beam listings are produced by running a list_beams script (e.g. list_beams_10941.py) in CASA which extracts the beam positions from each beam measurement set. This assumes the measurement sets have been recentred from their common position output by ASKAPSoft.

The catalogue is the continuum catalogue produced with Selavy as part of the ASKAPSoft pipeline processing.

### Output data

The votable (e.g. sb10941_srcs_image_params.vot) will list the combinations of the target and the beams to include. Each target will thus likely have multiple rows.

The csv (e.g. targets_10941.csv ) lists a single row for each target and includes a variable number of columns at the end, one for each beam to be included when processing that target.

In [2]:
# Parameters for the run
sbid = 13536
catalogue = 'sb{0}/selavy-image.i.SB13536.cont.restored.components.xml'.format(sbid)
beam_listing = 'sb{0}/beam_listing_SB{0}.csv'.format(sbid)
flux_peak_min = 15 # mJy/beam
max_sep_close = 0.55*u.deg # First pass maximum distance a beam is allowed to be from the target to be used.
max_sep_far = 0.8*u.deg # Fallback maximum distance a beam is allowed to be from the target to be used if none close

In [3]:
def rename_columns(table):
    names = np.asarray(table.colnames)
    for name in names:
        if name.startswith('col_'):
            table.rename_column(name, name[4:])

def slice_strings(a,start,end):
    if end is None:
        if start > 0:
            raise('end must be present when start is positive')
        b = a.view((str,1)).reshape(len(a),-1)[:,start:]
        return np.frombuffer(b.tostring(),dtype=(str,start*-1))
    
    b = a.view((str,1)).reshape(len(a),-1)[:,start:end]
    return np.frombuffer(b.tostring(),dtype=(str,end-start))


In [4]:
# Read and filter catalogue
src_votable = votable.parse(catalogue, pedantic=False)
table = src_votable.get_first_table().to_table()
rename_columns(table)
targets = table[table['flux_peak']>flux_peak_min]
targets

island_id,component_id,component_name,ra_hms_cont,dec_dms_cont,ra_deg_cont,dec_deg_cont,ra_err,dec_err,freq,flux_peak,flux_peak_err,flux_int,flux_int_err,maj_axis,min_axis,pos_ang,maj_axis_err,min_axis_err,pos_ang_err,maj_axis_deconv,min_axis_deconv,pos_ang_deconv,maj_axis_deconv_err,min_axis_deconv_err,pos_ang_deconv_err,chi_squared_fit,rms_fit_gauss,spectral_index,spectral_curvature,spectral_index_err,spectral_curvature_err,rms_image,has_siblings,fit_is_estimate,spectral_index_from_TT,flag_c4,comment
--,--,,,,deg,deg,arcsec,arcsec,MHz,mJy/beam,mJy/beam,mJy,mJy,arcsec,arcsec,deg,arcsec,arcsec,deg,arcsec,arcsec,deg,arcsec,arcsec,deg,--,mJy/beam,--,--,--,--,mJy/beam,,,,,
bytes20,bytes24,bytes26,bytes12,bytes13,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,bytes100
SB13536_island_1,SB13536_component_1a,J170413-421953,17:04:13.1,-42:19:53,256.054702,-42.331463,0.06,0.04,1419.5,308.815,2.533,1258.330,14.683,16.32,11.93,79.48,0.14,0.17,1.05,14.41,10.15,77.11,0.01,0.06,1.25,61378.711,10731.126,-99.00,-99.00,0.00,0.00,1.151,0,0,1,1,
SB13536_island_2,SB13536_component_2a,J164150-474038,16:41:50.5,-47:40:38,250.460518,-47.677297,0.03,0.02,1419.5,301.472,1.983,480.108,6.073,11.35,6.71,94.63,0.08,0.08,0.49,8.31,2.58,-84.23,0.02,0.39,0.65,3519.010,4497.131,-99.00,-99.00,0.00,0.00,1.324,0,0,1,1,
SB13536_island_3,SB13536_component_3a,J170247-440706,17:02:47.3,-44:07:06,255.696993,-44.118375,0.33,0.07,1419.5,217.568,17.959,264.925,25.838,8.54,6.82,93.05,0.28,0.32,2.24,3.64,2.85,-79.73,3.04,5.57,14.32,1017.076,2475.271,-99.00,-99.00,0.00,0.00,1.002,1,0,1,0,
SB13536_island_3,SB13536_component_3b,J170247-440706,17:02:47.8,-44:07:06,255.699238,-44.118508,0.34,0.07,1419.5,198.259,19.277,234.404,25.941,8.30,6.80,91.57,0.28,0.32,2.81,3.04,2.84,-83.97,18.25,21.57,57.16,1017.076,2475.271,-99.00,-99.00,0.00,0.00,1.002,1,0,1,0,
SB13536_island_4,SB13536_component_4a,J165326-420313,16:53:26.3,-42:03:13,253.359558,-42.053793,0.48,0.09,1419.5,232.115,22.026,322.273,36.733,9.43,7.04,87.02,0.42,0.42,2.78,5.42,3.32,81.98,0.57,2.20,6.44,2753.351,4174.479,-99.00,-99.00,0.00,0.00,0.883,1,0,1,1,
SB13536_island_4,SB13536_component_4b,J165325-420312,16:53:25.8,-42:03:12,253.357339,-42.053603,0.50,0.14,1419.5,159.016,28.620,205.216,38.945,8.58,7.19,77.93,0.34,0.40,7.69,4.32,2.94,40.63,1.04,2.95,23.01,2753.351,4174.479,-99.00,-99.00,0.00,0.00,0.883,1,0,1,1,
SB13536_island_5,SB13536_component_5a,J164151-463510,16:41:51.7,-46:35:10,250.465473,-46.586136,0.06,0.08,1419.5,181.984,2.679,991.542,16.729,21.90,11.89,143.09,0.21,0.20,0.41,20.82,9.47,-35.17,0.00,0.08,0.40,8840.620,2721.067,-99.00,-99.00,0.00,0.00,1.597,1,0,1,0,
SB13536_island_5,SB13536_component_5b,J164151-463503,16:41:51.8,-46:35:03,250.465687,-46.584312,0.09,0.15,1419.5,118.537,2.447,952.214,20.758,25.98,14.78,33.60,0.25,0.26,0.62,25.12,12.83,32.41,0.00,0.07,0.60,8840.620,2721.067,-99.00,-99.00,0.00,0.00,1.597,1,0,1,0,


In [15]:
# Build the list of beam locations
beams = ascii.read(beam_listing, format='no_header', guess=False, delimiter=',')
names = ('col1','col2', 'col3', 'col4')
new_names = ('filename','interleave', 'ra_rad', 'dec_rad')
beams.rename_columns(names, new_names)
print (beams[0])
#beams.pprint()

beam_id = slice_strings(beams['filename'], -8, -6)
print (beam_id)
#interleave_id = slice_strings(beams['interleave'], -1, None)# beams['interleave'][:][-1:]
interleave_id = slice_strings(beams['interleave'], -4, -3)# beams['interleave'][:][-1:]

print (interleave_id)
file_interleave = slice_strings(beams['filename'], -14, -13)
print (file_interleave)

ids = np.stack((beam_id, interleave_id), axis=-1)
unique_ids, unique_idx = np.unique(ids, axis=0, return_index=True)

beams['beam_id'] = beam_id
beams['interleave'] = interleave_id
beams['file_interleave'] = file_interleave
unique_beams = beams[beams['interleave'] == beams['file_interleave']]
print (unique_beams[0])

u_beam_locs = SkyCoord(ra=unique_beams['ra_rad']*u.rad, dec=unique_beams['dec_rad']*u.rad, frame='icrs')
u_beam_locs


                                                              filename                                                                   interleave        ra_rad        dec_rad    
------------------------------------------------------------------------------------------------------------------------------------ ----------------- ------------- ---------------
/avatar/nipingel/ASKAP/SMC/data/pilot_obs/ms_data/13531_13536/GASKAP_GP_HI_A/scienceData_SB13531_SB13536_GASKAP_GP_HI_A.beam00_SL.ms GASKAP_GP_HI_A_02 -1.9489433091 -0.826242361172
['00' '00' '00' '01' '01' '01' '02' '02' '02' '03' '03' '03' '04' '04'
 '04' '05' '05' '05' '06' '06' '06' '07' '07' '07' '08' '08' '08' '09'
 '09' '09' '10' '10' '10' '11' '11' '11' '12' '12' '12' '13' '13' '13'
 '14' '14' '14' '15' '15' '15' '16' '16' '16' '17' '17' '17' '18' '18'
 '18' '19' '19' '19' '20' '20' '20' '21' '21' '21' '22' '22' '22' '23'
 '23' '23' '24' '24' '24' '25' '25' '25' '26' '26' '26' '27' '27' '27'
 '28' '28' '28' '29' '29' '29' 

<SkyCoord (ICRS): (ra, dec) in deg
    [(248.33377388, -47.34020015), (249.66071719, -47.29794902),
     (250.98463758, -47.24091718), (252.30514991, -47.16916596),
     (253.62187967, -47.08275758), (254.9344648 , -46.98175486),
     (248.94550325, -46.54205701), (250.25192671, -46.49311434),
     (251.55527636, -46.42963683), (252.85520135, -46.35168321),
     (254.15136139, -46.25931252), (255.44342841, -46.15258385),
     (248.24928609, -45.78213094), (249.53877692, -45.74105671),
     (250.8254921 , -45.6856237 ), (252.1091036 , -45.61588779),
     (253.38929226, -45.53190485), (254.66574937, -45.43373049),
     (248.84409525, -44.98479495), (250.11469689, -44.93719342),
     (251.38248204, -44.87545431), (252.64715201, -44.79963087),
     (253.90841702, -44.70977581), (255.16599753, -44.60594114),
     (248.16685021, -44.224353  ), (249.42185081, -44.18438542),
     (250.67429119, -44.13043761), (251.92389246, -44.06256076),
     (253.17038322, -43.98080523), (254.41350087, -43.8

In [16]:
def get_beams_near_src(target_loc, beam_locs, beams, max_sep = 0.8*(1*u.deg)):
    """
    Find the beams within a certain angular distance of the target location.
    
    Parameters
    ----------
    target_loc: SkyCoord
        The sky location of the target.
    beam_locs: SkyCoord[]
        The locations of each beam.
    beams: str[]
        Array of beam names
    max_sep: dimension
        Maximum distance a beam centre can be from the target to be included (optional).
    
    Returns
    -------
    List of beam names within the requested distance and a list of their distances from the target.
    """
    beam_sep = beam_locs.separation(target_loc)
    beams_covering_target = beam_sep < max_sep
    src_beams = beams[beams_covering_target]
    src_beam_sep = beam_sep[beams_covering_target]
    return src_beams, src_beam_sep



In [17]:
comp_names = []
comp_ra = []
comp_dec = []
included_beam_nums = []
included_beam_interleaves = []
included_beam_ids = []
included_beam_sep = []

for tgt in targets:
    target_loc = SkyCoord(ra=tgt['ra_deg_cont']*u.degree, dec=tgt['dec_deg_cont']*u.degree, frame='icrs')
    src_beams, src_beam_sep = get_beams_near_src(target_loc, u_beam_locs, unique_beams, max_sep=max_sep_close)
    if len(src_beams) == 0:
        print ("No beams close to {}, checking out to {}".format(tgt['component_name'], max_sep_far))
        src_beams, src_beam_sep = get_beams_near_src(target_loc, u_beam_locs, unique_beams, max_sep=max_sep_far)

    for i in range(len(src_beams)):
        comp_names.append(tgt['component_name'])
        comp_ra.append(tgt['ra_deg_cont'])
        comp_dec.append(tgt['dec_deg_cont'])
        included_beam_nums.append(src_beams['beam_id'].data[i])
        included_beam_interleaves.append(src_beams['interleave'].data[i])
        included_beam_ids.append(src_beams['beam_id'].data[i]+src_beams['interleave'].data[i])
        included_beam_sep.append(src_beam_sep.to(u.deg).value[i])

image_params = Table()
image_params['component_name'] = comp_names
image_params['comp_ra'] = comp_ra
image_params['comp_dec'] = comp_dec
image_params['beam_nums'] = included_beam_nums
image_params['beam_interleaves'] = included_beam_interleaves
image_params['beam_ids'] = included_beam_ids
image_params['beam_sep'] = included_beam_sep

image_params_vot = votable.from_table(image_params)
filename = "sb{}_srcs_image_params.vot".format(sbid)
votable.writeto(image_params_vot, filename)

print ("Produced VO table file {} with the following contents:".format(filename))

image_params_vot.get_first_table()


No beams close to J170413-421953, checking out to 0.8 deg
No beams close to J165326-420313, checking out to 0.8 deg
No beams close to J165325-420312, checking out to 0.8 deg
No beams close to J163040-470019, checking out to 0.8 deg
No beams close to J163256-475753, checking out to 0.8 deg
No beams close to J163256-475800, checking out to 0.8 deg
No beams close to J165245-421113, checking out to 0.8 deg
No beams close to J165816-473102, checking out to 0.8 deg
No beams close to J165817-473050, checking out to 0.8 deg
No beams close to J162934-441837, checking out to 0.8 deg
No beams close to J170630-440828, checking out to 0.8 deg
No beams close to J170451-451720, checking out to 0.8 deg
No beams close to J163018-445231, checking out to 0.8 deg
No beams close to J162914-454117, checking out to 0.8 deg
No beams close to J170354-432605, checking out to 0.8 deg
No beams close to J165658-421009, checking out to 0.8 deg
No beams close to J165700-421002, checking out to 0.8 deg
No beams close

<Table masked=True length=1115>
component_name  comp_ra    comp_dec  ... beam_ids       beam_sep     
    str14       float64    float64   ...   str3         float64      
-------------- ---------- ---------- ... -------- -------------------
J170413-421953 256.054702 -42.331463 ...      35C  0.6428115769373824
J164150-474038 250.460518 -47.677297 ...      01C  0.5457984257985384
J170247-440706 255.696993 -44.118375 ...      23B 0.31034406576444123
J170247-440706 255.696993 -44.118375 ...      23C 0.30817082832100157
J170247-440706 255.699238 -44.118508 ...      23B 0.31196123450978136
J170247-440706 255.699238 -44.118508 ...      23C 0.30751386653661755
J165326-420313 253.359558 -42.053793 ...      34B    0.67278503248002
J165325-420312 253.357339 -42.053603 ...      34B   0.673730807716681
J164151-463510 250.465473 -46.586136 ...      07A 0.17386546107849704
J164151-463510 250.465473 -46.586136 ...      01B   0.489169246926938
           ...        ...        ... ...      ...         

In [18]:
print ("Count of the number of sources using each beam")
ar = np.array(included_beam_ids)
for i in range(36):
    for interleave in ('A', 'B', 'C'):
        key = '{:02d}{}'.format(i, interleave)
        count = len(ar[ar==key])
        print (key, count)
print ('mean targets per beam {:.2f}'.format(len(ar)/(36*3)))
mean_beams_per_source = len(ar) / len(targets)
print ('mean beams per target {:.2f}'.format(mean_beams_per_source))

Count of the number of sources using each beam
00A 22
00B 9
00C 30
01A 30
01B 47
01C 28
02A 14
02B 19
02C 7
03A 9
03B 8
03C 10
04A 12
04B 8
04C 8
05A 11
05B 7
05C 7
06A 8
06B 3
06C 31
07A 42
07B 27
07C 32
08A 8
08B 16
08C 9
09A 10
09B 3
09C 4
10A 2
10B 6
10C 1
11A 6
11B 9
11C 18
12A 4
12B 7
12C 1
13A 5
13B 8
13C 8
14A 16
14B 14
14C 22
15A 17
15B 9
15C 10
16A 9
16B 6
16C 10
17A 2
17B 5
17C 4
18A 7
18B 6
18C 8
19A 6
19B 14
19C 8
20A 9
20B 12
20C 4
21A 9
21B 9
21C 5
22A 2
22B 9
22C 8
23A 5
23B 7
23C 6
24A 8
24B 4
24C 4
25A 5
25B 8
25C 14
26A 15
26B 12
26C 12
27A 6
27B 10
27C 4
28A 8
28B 8
28C 10
29A 10
29B 7
29C 9
30A 5
30B 6
30C 7
31A 6
31B 2
31C 1
32A 4
32B 4
32C 9
33A 9
33B 5
33C 7
34A 8
34B 17
34C 11
35A 17
35B 21
35C 10
mean targets per beam 10.32
mean beams per target 3.20


In [19]:
ar = np.array(included_beam_ids)
print ('Count of total beam usage={}\nNumber using beam 02A={}'.format(len(ar), len(ar[ar=='02A'])))

Count of total beam usage=1115
Number using beam 02A=14


In [20]:
print ('Example source and included beams')
tgt = targets[100]
target_loc = SkyCoord(ra=tgt['ra_deg_cont']*u.degree, dec=tgt['dec_deg_cont']*u.degree, frame='icrs')
src_beams, src_beam_sep = get_beams_near_src(target_loc, u_beam_locs, unique_beams)
print(tgt['component_name'])
print(tgt['ra_deg_cont'])
print(tgt['dec_deg_cont'])
print(src_beams['beam_id'].data)

Example source and included beams
J163830-470036
249.626546
-47.010183
['01' '06' '07' '00' '01' '00' '01' '06']


In [21]:
# Create the csv file of targets and included beams
csv_filename = 'targets_{}.csv'.format(sbid)
with open(csv_filename, 'w') as csvfile:
    tgtwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    tgtwriter.writerow(['index', 'component_name', 'ra', 'dec', 'beams'])
    i = 1;
    for tgt in targets:
        comp_name = tgt['component_name']
        row = []
        row.append(str(i))
        row.append(comp_name)
        row.append(tgt['ra_deg_cont'])
        row.append(tgt['dec_deg_cont'])
        for tgt_beam in image_params:
            if comp_name == tgt_beam['component_name']:
                row.append(tgt_beam['beam_ids'])
        tgtwriter.writerow(row)
        i+= 1

print ("Produced VO table file {} with {} target beam combos.".format(filename, len(image_params_vot.get_first_table().array)))
print ('Produced csv file {} with {} targets (mean {:.2f} beams/target).'.format(csv_filename, i, mean_beams_per_source))

Produced VO table file sb13536_srcs_image_params.vot with 1115 target beam combos.
Produced csv file targets_13536.csv with 349 targets (mean 3.20 beams/target).
